# Kaggle's tutorial to natural language processing
In this notebook I will follow the tutorial found at https://www.kaggle.com/code/philculliton/nlp-getting-started-tutorial.

In [1]:
import numpy as np
import pandas as pd
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [3]:
train_df = pd.read_csv("../Datasets/nlp-getting-started/train.csv")
test_df = pd.read_csv("../Datasets/nlp-getting-started/test.csv")

What does out data look like?

In [37]:
print(train_df.columns)
train_df.loc[100:120, :]

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')


,id,keyword,location,text,target
100,144,accident,UK,.@NorwayMFA #Bahrain police had previously die...,1
101,145,accident,"Nairobi, Kenya",I still have not heard Church Leaders of Kenya...,0
102,146,aftershock,Instagram - @heyimginog,@afterShock_DeLo scuf ps live and the game... cya,0
103,149,aftershock,304,'The man who can drive himself further once th...,0
104,151,aftershock,Switzerland,320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/yN...,0
105,153,aftershock,304,'There is no victory at bargain basement price...,0
106,156,aftershock,US,320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/vA...,0
107,157,aftershock,304,'Nobody remembers who came in second.' Charles...,0
108,158,aftershock,Instagram - @heyimginog,@afterShock_DeLo im speaking from someone that...,0
109,159,aftershock,304,'The harder the conflict the more glorious the...,0


So, there are 4 explanatory variables: `id` is a simple ID that will clearly be removed; `keyword` relates to the word that was picked up by the algorithm that needs to classify tweets (or similar); `location` is sometimes a place, sometimes a number, sometimes another reference (_e.g_ Instagram); finally, `text` refers to the actual text. The response variable is `target`, which is either 1 (for a disaster tweet) or 0 (for an unrelated tweet.)